# news_classification

In [107]:
import pandas as pd
import re
from collections import Counter
from hazm import Normalizer, word_tokenize, stopwords_list
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, classification_report
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

## Load_dataset

In [2]:
df_train = pd.read_csv("./data/train_data.csv")
df_train.head()

,title,description,tags
0,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...,استانها
1,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...,استانها
2,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,گروه استان‌ها ــ با اعلام شیوع ویروس کرونا در ...,استانها
3,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...,یک متخصص بیوتکنولوژی پزشکی با بیان اینکه مرگ‌و...,اجتماعی
4,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,رئیس اتحادیه فروشگاه‌های زنجیره‌ای با اشاره به...,اقتصادی


## EDA

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12457 entries, 0 to 12456
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        12457 non-null  object
 1   description  12457 non-null  object
 2   tags         12455 non-null  object
dtypes: object(3)
memory usage: 292.1+ KB


In [4]:
df_train.shape

(12457, 3)

### Missing value

In [5]:
df_train.isna().sum()

title          0
description    0
tags           2
dtype: int64

In [6]:
df_train[df_train["tags"].isna()]

,title,description,tags
11659,مجید آذرپی به زندان اوین بازگشت,مجید آذرپی، زندانی سیاسی اصلاح‌طلب و عضو ستاد ...,NaN
12186,مقام آموزش و پرورش: الزام دانش آموزان به پوشش ...,مدیرکل انجمن اولیا و مربیان وزارت آموزش و پرور...,NaN


In [7]:
df_train.dropna(inplace=True)

In [8]:
df_train[["description", "title"]].agg(
    lambda x: x.str.len()
    .agg(
        ["mean", "min", "max", "median", "count", "std"]
    )
).round(2)

,description,title
mean,2406.14,62.86
min,1.00,11.00
max,55609.00,187.00
median,1218.00,60.00
count,12455.00,12455.00
std,2952.85,19.42


In [9]:
df_train["description"].str.len().sort_values()[:15]

9851      1
10038     1
9756      3
9752      3
9755      3
322      43
559      44
1381     44
200      45
551      46
337      47
1587     49
2930     51
140      53
1316     54
Name: description, dtype: int64

In [10]:
df_train["description"][df_train["description"].str.len().isin([1, 3])]

9752     . \n
9755     . \n
9756     . \n
9851       \n
10038      \n
Name: description, dtype: object

In [11]:
invalid_description_index = df_train["description"][df_train["description"].str.len().isin([
    1, 3])].index
invalid_description_index

Index([9752, 9755, 9756, 9851, 10038], dtype='int64')

In [12]:
df_train.drop(invalid_description_index,inplace=True)

In [13]:
df_train.shape

(12450, 3)

### Create Text data form 'title' and 'description'

In [14]:
df_train["text"] = df_train["title"] + " " + df_train["description"].str.strip()
df_train.drop(columns=["title","description"],inplace=True)
df_train

,tags,text
0,استانها,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...
1,استانها,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...
2,استانها,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...
3,اجتماعی,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...
4,اقتصادی,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...
...,...,...
12452,سیاسی,علیرضا زاکانی رئیس مرکز پژوهش‌های مجلس شد به ن...
12453,سیاسی,قوه قضائیه: تقاضای فرجام در پرونده روح‌الله زم...
12454,بایگانی,روسیه: ابراز نگرانی لاوروف و ظریف درباره حضور ...
12455,بایگانی,وزیر خارجه ایران وارد بیروت شد در حالی که اعتر...


In [15]:
df_train["tags"].value_counts()

tags
بایگانی                 1943
جهان                    1657
ایران                   1188
بین الملل                972
سیاسی                    921
                        ... 
دوره‌های زبان آلمانی       1
الشرق الأوسط               1
بازار                      1
ویدئو > گزارش              1
دویچه وله                  1
Name: count, Length: 100, dtype: int64

##### remove unnesesery column type 

In [16]:
df_train = df_train[df_train["tags"]!="بایگانی"]

In [17]:
df_train.head()

,tags,text
0,استانها,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...
1,استانها,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...
2,استانها,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...
3,اجتماعی,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...
4,اقتصادی,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...


### Text Cleaning

In [18]:
df_train[df_train["text"].str.contains("&quot")]

,tags,text
3,اجتماعی,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...
18,سیاسی,بررسی صلاحیت &quot;خاوازی&quot; در کمیسیون کشا...
33,اجتماعی,تقلا برای ایجاد ناامنی روانی در جامعه؛‌ ویروس ...
77,اجتماعی,&quot;تلویزیون&quot; مدرسه دانش‌آموزان شد آموز...
85,سیاسی,روحانی : همه دولت‌های جهان باید برای مقابله &q...
...,...,...
6426,ایران,"""۶۲ درصد اقلیت‌های جنسی قربانی خشونت در خانواد..."
6427,جهان,۷۵مین سالروز هیروشیما: فاجعه‌ای که هرگز نباید ...
6428,آلمان,۷۵مین سالگرد پایان جنگ جهانی دوم؛ بحث ها بر سر...
6429,آلمان,۷۵مین سالگرد پایان جنگ جهانی دوم؛ روز رهایی یا...


In [19]:
df_train.loc[:,"text"]=df_train.loc[:,"text"].str.replace("&quot"," ")

In [20]:
df_train.head()

,tags,text
0,استانها,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...
1,استانها,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...
2,استانها,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...
3,اجتماعی,واکنش ;پروفسور کرمی ; به دروغ‌پراکنی برخی رسا...
4,اقتصادی,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...


In [21]:
PERSIAN_MAP = str.maketrans({
    "ك": "ک", "ي": "ی",
    "ة": "ه", "ؤ": "و", "إ": "ا", "أ": "ا", "ٱ": "ا"
})
html_tag_re = re.compile(r"<[^>]+>")
extra_space_re = re.compile(r"\s+")
url_re = re.compile(r"https?://\S+|www\.\S+")
mention_re = re.compile(r"[@#]\S+")
digit_re = re.compile(r"\d+")

In [35]:
def clean_text(s:str)->str:
    s = str(s)
    s=s.translate(PERSIAN_MAP)
    s = html_tag_re.sub(" ",s)
    s = url_re.sub(" ",s)
    s = mention_re.sub(" ",s)
    s = digit_re.sub(" ",s)
    s = re.sub(r"[^\w\s\u0600-\u06FF]"," ",s)
    s = extra_space_re.sub(" ",s).strip()
    return s

df_train.loc[:,"clean_text"] = df_train["text"].map(clean_text)

In [23]:
df_train.head()

,tags,text,clean_text
0,استانها,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,روایتی از تحصیل زیرسقف های لرزان و سرمای سوزان...
1,استانها,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,انقلابی سرشناس بحرین در گفت وگو با تسنیم مردم ...
2,استانها,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,زندگی مردم در شهر قم جریان دارد پیشگیری در اول...
3,اجتماعی,واکنش ;پروفسور کرمی ; به دروغ‌پراکنی برخی رسا...,واکنش پروفسور کرمی به دروغ پراکنی برخی رسانه ه...
4,اقتصادی,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,مردم نگران تامین کالاهای موردنیاز خود نباشند م...


In [24]:
normalizer = Normalizer()
stopwords = set(stopwords_list())


def clean_text_hazm(s:str)->str:
    s = clean_text(s)
    s = normalizer.normalize(s)
    tokens = [t for t in word_tokenize(s) if t not in stopwords and len(t) > 1]
    return " ".join(tokens)



In [34]:
df_train.loc[:,"final_text"]= df_train["clean_text"].map(clean_text_hazm)

In [26]:
df_train.head()

,tags,text,clean_text,final_text
0,استانها,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,روایتی از تحصیل زیرسقف های لرزان و سرمای سوزان...,روایتی تحصیل زیرسقف‌های لرزان سرمای سوزان دانش...
1,استانها,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,انقلابی سرشناس بحرین در گفت وگو با تسنیم مردم ...,انقلابی سرشناس بحرین گفت‌وگو تسنیم مردم منطقه ...
2,استانها,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,زندگی مردم در شهر قم جریان دارد پیشگیری در اول...,زندگی مردم شهر قم پیشگیری اولویت مردم قرار تصا...
3,اجتماعی,واکنش ;پروفسور کرمی ; به دروغ‌پراکنی برخی رسا...,واکنش پروفسور کرمی به دروغ پراکنی برخی رسانه ه...,واکنش پروفسور کرمی دروغ‌پراکنی رسانه‌ها القای ...
4,اقتصادی,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,مردم نگران تامین کالاهای موردنیاز خود نباشند م...,مردم نگران تامین کالاهای موردنیاز نباشند میزان...


In [29]:
Counter(" ".join(df_train["final_text"]).split()).most_common(20)

[('zwnj', 57061),
 ('می\u200czwnj', 28057),
 ('ایران', 24762),
 ('zwnj\u200cهای', 23330),
 ('کرونا', 22220),
 ('کشور', 18473),
 ('آمریکا', 13843),
 ('سال', 13635),
 ('zwnj\u200cها', 12181),
 ('روز', 11250),
 ('دولت', 10692),
 ('ویروس', 10008),
 ('مردم', 9939),
 ('هزار', 8944),
 ('گزارش', 8898),
 ('قرار', 8882),
 ('نفر', 8760),
 ('سازمان', 8366),
 ('اعلام', 8291),
 ('اسلامی', 7898)]

#### remove \u200c , \u200d after normalize and remove stop word they comes

In [33]:
def clean_zwnj(text: str) -> str:
    # حذف نیم‌فاصله‌ی یونیکد
    text = text.replace("\u200c", "")
    text = text.replace("\u200d", "")
    # حذف رشته‌ی "zwnj" اگر به‌صورت متن مونده
    text = text.replace("zwnj", "")
    return text

df_train.loc[:, "final_text"] = df_train["final_text"].map(clean_zwnj)

In [32]:
df_train.head()

,tags,text,clean_text,final_text
0,استانها,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,روایتی از تحصیل زیرسقف های لرزان و سرمای سوزان...,روایتی تحصیل زیرسقفهای لرزان سرمای سوزان دانش ...
1,استانها,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,انقلابی سرشناس بحرین در گفت وگو با تسنیم مردم ...,انقلابی سرشناس بحرین گفتوگو تسنیم مردم منطقه پ...
2,استانها,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,زندگی مردم در شهر قم جریان دارد پیشگیری در اول...,زندگی مردم شهر قم پیشگیری اولویت مردم قرار تصا...
3,اجتماعی,واکنش ;پروفسور کرمی ; به دروغ‌پراکنی برخی رسا...,واکنش پروفسور کرمی به دروغ پراکنی برخی رسانه ه...,واکنش پروفسور کرمی دروغپراکنی رسانهها القای تر...
4,اقتصادی,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,مردم نگران تامین کالاهای موردنیاز خود نباشند م...,مردم نگران تامین کالاهای موردنیاز نباشند میزان...


#### Working on tags labels

In [36]:
df_train["tags"]

0        استانها
1        استانها
2        استانها
3        اجتماعی
4        اقتصادی
          ...   
12429    خبرخوان
12441      سیاسی
12447    اجتماعی
12452      سیاسی
12453      سیاسی
Name: tags, Length: 10507, dtype: object

In [37]:
def norm_tag(s):
    if pd.isna(s): return ""
    s = str(s)
    s = s.replace("ي","ی").replace("ك","ک").replace("\u200c","")  # ی/ک و حذف نیم‌فاصله
    s = re.sub(r"\s+", " ", s).strip()
    return s

df_train["tag_clean"] = df_train["tags"].map(norm_tag)

/tmp/ipykernel_24427/79702925.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["tag_clean"] = df_train["tags"].map(norm_tag)


In [38]:
df_train["tag_clean"]

0        استانها
1        استانها
2        استانها
3        اجتماعی
4        اقتصادی
          ...   
12429    خبرخوان
12441      سیاسی
12447    اجتماعی
12452      سیاسی
12453      سیاسی
Name: tag_clean, Length: 10507, dtype: object

In [39]:
uniq = df_train["tag_clean"].value_counts()
print(uniq.head(30))
print("Unique tags:", uniq.shape[0])

tag_clean
جهان                                    1657
ایران                                   1208
بین الملل                                972
سیاسی                                    921
سیاسی > دولت                             506
اجتماعی                                  493
اقتصادی                                  478
استانها                                  433
سیاسی > سیاست خارجی                      418
اجتماعی > سلامت                          407
آلمان                                    310
سیاسی > مجلس                             266
سیاسی > سیاست داخلی                      261
دانش و محیط زیست                         203
سیاسی > حقوقی و قضایی                    166
اقتصاد                                   133
فرهنگی                                   126
سیاسی > دفاعی - امنیتی                   110
اقتصادی > اقتصاد کلان                    108
اجتماعی > حوادث، انتظامی                  79
فرهنگ و هنر                               75
ورزشی > فوتبال، فوتسال                    68


In [40]:
rules = {
    "بین الملل": ["بین الملل", "جهان", "آلمان", "اروپا", "آمریکا"],
    "ایران_استانها": ["استان", "استانها", "ایران"],
    "سیاسی": ["سیاسی", "سیاست", "مجلس", "انتخابات", "قوه", "رییس جمهور", "رئیس جمهور", "دولت"],
    "اقتصادی": ["اقتصاد", "اقتصادی", "بورس", "بانک", "ارز", "تورم", "نفت", "سهام", "بازار", "سرمایه"],
    "علمی_فرهنگی_ورزشی": [
        "علمی", "دانشگاه", "فرهنگی", "فرهنگ", "هنر", "سینما", "کتاب", "پژوهش",
        "ورزشی", "فوتبال", "کشتی", "والیبال", "توپ و تور", "بسکتبال", "تکواندو", "گردشگری", "میراث"
    ]
}

def map_to_six(tag: str) -> str:
    if not isinstance(tag, str):
        return "اجتماعی"   # پیش‌فرض
    
    for label, keywords in rules.items():
        if any(kw in tag for kw in keywords):
            return label
    return "اجتماعی"       # اگر هیچ قانونی نخورد

In [42]:
df_train.loc[:,"label6"] = df_train["tag_clean"].map(map_to_six)
print(df_train["label6"].value_counts())

label6
بین الملل            3045
سیاسی                2717
ایران_استانها        1863
اجتماعی              1492
اقتصادی               884
علمی_فرهنگی_ورزشی     506
Name: count, dtype: int64


In [44]:
label_to_id = {
    "اجتماعی": 0,
    "اقتصادی": 1,
    "ایران_استانها": 2,
    "بین الملل": 3,
    "سیاسی": 4,
    "علمی_فرهنگی_ورزشی": 5
}
df_train.loc[:,"y"] = df_train["label6"].map(label_to_id)

In [46]:
df_train.head()

,tags,text,clean_text,final_text,tag_clean,label6,y
0,استانها,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,روایتی از تحصیل زیرسقف های لرزان و سرمای سوزان...,روایتی تحصیل زیرسقف‌های لرزان سرمای سوزان دانش...,استانها,ایران_استانها,2
1,استانها,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,انقلابی سرشناس بحرین در گفت وگو با تسنیم مردم ...,انقلابی سرشناس بحرین گفت‌وگو تسنیم مردم منطقه ...,استانها,ایران_استانها,2
2,استانها,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,زندگی مردم در شهر قم جریان دارد پیشگیری در اول...,زندگی مردم شهر قم پیشگیری اولویت مردم قرار تصا...,استانها,ایران_استانها,2
3,اجتماعی,واکنش ;پروفسور کرمی ; به دروغ‌پراکنی برخی رسا...,واکنش پروفسور کرمی به دروغ پراکنی برخی رسانه ه...,واکنش پروفسور کرمی دروغ‌پراکنی رسانه‌ها القای ...,اجتماعی,اجتماعی,0
4,اقتصادی,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,مردم نگران تامین کالاهای موردنیاز خود نباشند م...,مردم نگران تامین کالاهای موردنیاز نباشند میزان...,اقتصادی,اقتصادی,1


In [48]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train["final_text"], df_train["y"],
    test_size=0.2,    
    random_state=42,
    stratify=df_train["y"]  
)


In [ ]:

pipeline = Pipeline([
    ("tf_idf" , TfidfVectorizer(
        analyzer="word",
        ngram_range=(1,2),
        max_features=15000,    
        min_df=5,              
        max_df=0.95,           
        sublinear_tf=True
    )),
    ("model",LinearSVC())
])

param_grid = {
    "model__C":[0.01,0.05,0.1,0.2],
    "tf_idf__min_df":[5,10,15]
}

grid = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    scoring="f1_weighted",   
    cv=3,                    
    n_jobs=-1,              
    verbose=1
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best CV F1:", grid.best_score_)

# ارزیابی روی Validation
pred_val = grid.predict(X_test)
print("Validation F1 (weighted):", f1_score(y_test, pred_val, average="weighted"))
print(classification_report(y_test, pred_val))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
Best params: {'model__C': 0.2, 'tf_idf__min_df': 15}
Best CV F1: 0.8265274331001916
Validation F1 (weighted): 0.8399716361256102
              precision    recall  f1-score   support

           0       0.80      0.69      0.74       298
           1       0.81      0.72      0.76       177
           2       0.91      0.82      0.86       373
           3       0.84      0.91      0.88       609
           4       0.83      0.93      0.88       544
           5       0.81      0.68      0.74       101

    accuracy                           0.84      2102
   macro avg       0.84      0.79      0.81      2102
weighted avg       0.84      0.84      0.84      2102



In [131]:
pred_val_train = grid.predict(X_train)
print("Train validation F1 (weighted):", f1_score(y_train, pred_val_train, average="weighted"))
print(classification_report(y_train, pred_val_train))

Train validation F1 (weighted): 0.9434296558836581
              precision    recall  f1-score   support

           0       0.95      0.87      0.91      1194
           1       0.94      0.93      0.93       707
           2       0.96      0.92      0.94      1490
           3       0.94      0.97      0.95      2436
           4       0.95      0.99      0.97      2173
           5       0.93      0.90      0.91       405

    accuracy                           0.94      8405
   macro avg       0.94      0.93      0.94      8405
weighted avg       0.94      0.94      0.94      8405

